In [177]:
%load_ext cudf.pandas

In [178]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [118]:
def get_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content,'html.parser')
    return soup,r

In [199]:
def get_anime_details(soup):
    details = {}
    details['name'] = soup.find(class_='title-name').text
    details['description'] = soup.find(itemprop='description').text
    details['score']=soup.find(itemprop='ratingValue').text
    info_list = [i.text.replace('\n','').replace(' ','') for i in 
                 soup.find(class_='leftside').find_all(class_='spaceit_pad')]

    cols = ['Type', 'Episodes','Premiered','Studios', 'Source', 
            'Genres','Rating','Members', 'Favorites']

    for col in cols:
        details[col.lower()] = next((element.split(':')[-1] for element 
                             in info_list if col in element),None)

    details['genres'] = ','.join(map(str,[i[:len(i)//2] for i in details['genres'].split(',')]))
    return pd.DataFrame(details,index=[0])

In [184]:
def get_review_detail(review_soup):
    details = {}
    details['username'] = review_soup.find(class_='username').text.replace('\n','')
    details['anime_name'] = review_soup.find(class_='title').text
    details['anime_link'] = review_soup.find(class_='title')['href']
    details['recommended'] = review_soup.find(class_='tag').text
    details['rating_user'] = review_soup.find(class_='rating').find(class_='num').text
    return pd.DataFrame(details,index=[0])

In [194]:
df = pd.DataFrame()

In [196]:
page = 69

In [197]:
url = f'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=2&preliminary=off&spoiler=on&p={page}'
soup,_ = get_soup(url)
all_reviews = soup.find_all(class_='review-element js-review-element')
for review in all_reviews:
    df = pd.concat([df,get_review_detail(review)],ignore_index=True)

In [203]:
df.head()

,username,anime_name,anime_link,recommended,rating_user
0,Ruchan68,Zom 100: Zombie ni Naru made ni Shitai 100 no ...,https://myanimelist.net/anime/54112/Zom_100__Z...,Recommended,10
1,TheEtherea,Rokka no Yuusha,https://myanimelist.net/anime/28497/Rokka_no_Y...,Not Recommended,4
2,Kays90,The Legend of Heroes: Sen no Kiseki - Northern...,https://myanimelist.net/anime/48441/The_Legend...,Recommended,8
3,bergars,Omoide Poroporo,https://myanimelist.net/anime/1029/Omoide_Poro...,Recommended,8
4,Phantom_373,Aura: Maryuuin Kouga Saigo no Tatakai,https://myanimelist.net/anime/14669/Aura__Mary...,Recommended,10


In [202]:
soup,_ = get_soup(df.iloc[0,2])
get_anime_details(soup)

,name,description,score,type,episodes,premiered,studios,source,genres,rating,members,favorites
0,Zom 100: Zombie ni Naru made ni Shitai 100 no ...,After graduating from a top university with an...,7.83,TV,12,Summer2023,BUGFILMS,Manga,"Action,Comedy,Horror,Supernatural,Suspense",R-17+(violence&profanity),"517,604","3,858"
